# Generate JSONL of facts and fake context facts using PopQA

In [2]:
import pandas as pd
import json
import numpy as np
import random
from ast import literal_eval
import json
from datasets import load_dataset
import os
os.chdir('..')
print(os.getcwd())
from utils.io import write_dicts_one_by_one

random.seed(42)
np.random.seed(42)

/home/ckuempel/Assertions/.venv_asser/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/ckuempel/Assertions


In [17]:
ds = load_dataset("akariasai/PopQA")
df = ds["test"].to_pandas()

Repo card metadata block was not found. Setting CardData to empty.


In [18]:
print(f"Original dataset size: {len(df)}")
print(f"Relations: {df['prop'].value_counts()}")

Original dataset size: 14267
Relations: prop
director          1999
screenwriter      1999
genre             1619
producer          1520
author            1514
composer           978
country            838
capital            645
place of birth     584
father             570
sport              547
occupation         532
capital of         363
religion           338
mother             187
color               34
Name: count, dtype: int64


### EDA

In [19]:
# Sample 10 rows from each prop and display subj, obj, and prop
for prop in df["prop"].unique():
    print(f"\n=== {prop} ===")
    prop_subset = df[df["prop"] == prop]
    sample_size = min(10, len(prop_subset))
    sampled_rows = prop_subset.sample(n=sample_size, random_state=42)
    print(sampled_rows[["subj", "obj", "prop"]].to_string(index=False))



=== occupation ===
                subj            obj       prop
       Scooter Braun talent manager occupation
   Virginia Haviland      librarian occupation
       Annie Beustes     politician occupation
          Ali Hillis          actor occupation
   Guy Joseph Bonnet  major general occupation
      Eliza Lawrence     politician occupation
         Alan Huckle       diplomat occupation
Charles Martin Smith          actor occupation
      Jörg A. Eggers   screenwriter occupation
          Jon Eikemo          actor occupation

=== place of birth ===
              subj          obj           prop
      Paul Braniff   Portaferry place of birth
Alexander Butlerov       Çistay place of birth
      Mike Gregory         Bath place of birth
 Christian Vaquero   Montevideo place of birth
 Alexandros Matsas       Athens place of birth
    Charles Plumer  Canons Park place of birth
       Bob Warlick      Hickory place of birth
       Terry Hunte Saint Philip place of birth
       James Bro

### Filtering/Preprocessing

In [20]:
# filter out capital of because that one is weird
df = df[df["prop"] != "capital of"]
df

,id,subj,prop,obj,subj_id,prop_id,obj_id,s_aliases,o_aliases,s_uri,o_uri,s_wiki_title,o_wiki_title,s_pop,o_pop,question,possible_answers
0,4222362,George Rankin,occupation,politician,1850297,22,2834605,"[""George James Rankin""]","[""political leader"",""political figure"",""polit....",http://www.wikidata.org/entity/Q5543720,http://www.wikidata.org/entity/Q82955,George Rankin,Politician,142,25692,What is George Rankin's occupation?,"[""politician"", ""political leader"", ""political ..."
1,4725190,John Mayne,occupation,journalist,2079053,22,663400,[],"[""journo"",""journalists""]",http://www.wikidata.org/entity/Q6247345,http://www.wikidata.org/entity/Q1930187,John Mayne,Journalist,236,24952,What is John Mayne's occupation?,"[""journalist"", ""journo"", ""journalists""]"
2,4382392,Henry Feilden,occupation,politician,1925450,22,2834605,"[""Henry Master Feilden""]","[""political leader"",""political figure"",""polit....",http://www.wikidata.org/entity/Q5725578,http://www.wikidata.org/entity/Q82955,Henry Feilden (Conservative politician),Politician,58,25692,What is Henry Feilden's occupation?,"[""politician"", ""political leader"", ""political ..."
3,4822110,Kathy Saltzman,occupation,politician,2122743,22,2834605,[],"[""political leader"",""political figure"",""polit....",http://www.wikidata.org/entity/Q6377295,http://www.wikidata.org/entity/Q82955,Kathy Saltzman,Politician,127,25692,What is Kathy Saltzman's occupation?,"[""politician"", ""political leader"", ""political ..."
4,4011112,Eleanor Davis,occupation,cartoonist,1752619,22,68412,"[""Eleanor McCutcheon Davis""]","[""graphic artist"",""animator"",""illustrator""]",http://www.wikidata.org/entity/Q5354261,http://www.wikidata.org/entity/Q1114448,Eleanor Davis,Cartoonist,317,9649,What is Eleanor Davis's occupation?,"[""cartoonist"", ""graphic artist"", ""animator"", ""..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14262,4165948,Netherlands,capital,Amsterdam,1825504,422,2455815,"[""Holland"",""the Netherlands"",""NL"",""NED"",""Neder...","[""Mokum"",""Amsterdam, NL"",""Amsterdam, Netherlan...",http://www.wikidata.org/entity/Q55,http://www.wikidata.org/entity/Q727,Netherlands,Amsterdam,416350,147710,What is the capital of Netherlands?,"[""The Hague"", ""Den Haag"", ""'s-Gravenhage"", ""Ha..."
14263,2617063,City Municipality of Celje,capital,Celje,1127153,422,5035,"[""Celje city municipality""]",[],http://www.wikidata.org/entity/Q3441823,http://www.wikidata.org/entity/Q1012,City Municipality of Celje,Celje,335,4086,What is the capital of City Municipality of Ce...,"[""Celje""]"
14264,471864,Delhi,capital,New Delhi,192624,422,2938512,"[""National Capital Region of Delhi"",""NCR""]","[""New Delhi district"",""Nayi Dilli""]",http://www.wikidata.org/entity/Q1353,http://www.wikidata.org/entity/Q987,Delhi,New Delhi,219227,128193,What is the capital of Delhi?,"[""New Delhi"", ""New Delhi district"", ""Nayi Dilli""]"
14265,2965573,Palestinian territories,capital,Jerusalem,1270077,422,127625,"[""occupied Palestinian territories"",""OPT"",""Wes...","[""Yerushalayim"",""J'lem"",""Aelia Capitolina"",""Al...",http://www.wikidata.org/entity/Q407199,http://www.wikidata.org/entity/Q1218,Palestinian territories,Jerusalem,17545,136734,What is the capital of Palestinian territories?,"[""Jerusalem"", ""Yerushalayim"", ""J'lem"", ""Aelia ..."


In [21]:
# Remove rows with missing or empty answers
df = df.dropna(subset=['obj', 'subj'])
df = df[df['obj'].str.strip() != '']
df = df[df['subj'].str.strip() != '']
print(f"After removing empty answers: {len(df)}")

After removing empty answers: 13901


In [22]:
# Remove rows where the answer is too ambiguous.
print(df["prop"].value_counts())
print(df[df["subj"] == "Pilot"].head())
print(df[["subj", "prop"]].value_counts())


# Filter out rows where the combination of subj and prop appears more than once
subj_prop_counts = df.groupby(["subj", "prop"]).size()
duplicated_pairs = subj_prop_counts[subj_prop_counts > 1].index
df_filtered = df.set_index(["subj", "prop"]).drop(duplicated_pairs).reset_index()
df = df_filtered
print("After filtering, subj-prop pairs should be unique:")
print(df[["subj", "prop"]].value_counts()) ### All should be 1
print(df[df["subj"] == "London"]) # make sure no duplicate subj-prop pairs

print("After filtering, obj-prop pairs need not be unique:")
print(df[["obj", "prop"]].value_counts()) # obj-prop pairs need not be unique
print(df[df["obj"] == "association football"]) 

print(f"After removing: {len(df)}")


prop
director          1999
screenwriter      1998
genre             1619
producer          1520
author            1513
composer           978
country            838
capital            644
place of birth     584
father             570
sport              547
occupation         532
religion           338
mother             187
color               34
Name: count, dtype: int64
           id   subj      prop               obj  subj_id  prop_id   obj_id  \
5763  5453595  Pilot  director  Francis Lawrence  2425271      526  1883124   
5805  5453492  Pilot  director        Peter Berg  2425233      526   607993   
5818  4577696  Pilot  director     Marc Buckland  2018998      526  1230313   
5932  2957316  Pilot  director     Phillip Noyce  1266576      526  1791491   
5957   149939  Pilot  director      Brett Ratner    60507      526  1052983   

                        s_aliases o_aliases  \
5763  ["Tales of the Red Thread"]        []   
5805                           []        []   
5818    

subj               prop        
Pilot              screenwriter    40
                   director        22
Mother             director         8
Click              author           6
Earth              screenwriter     5
                                   ..
Higher Learning    producer         1
                   screenwriter     1
Higher and Higher  genre            1
Highland           capital          1
∞                  producer         1
Name: count, Length: 12810, dtype: int64
After filtering, subj-prop pairs should be unique:
subj                 prop      
"Them"               producer      1
Ruth Berman Harris   genre         1
Ruslan and Lyudmila  composer      1
Russell Stokes       occupation    1
Russia               capital       1
                                  ..
Hara                 genre         1
Harald Molander      mother        1
Harbor Lights        composer      1
Hard                 producer      1
∞                    producer      1
Name: count, Length

In [23]:
def is_bijective(df, col_a, col_b):
    # Check if A->B mapping is one-to-one
    a_to_b_unique = df.groupby(col_a)[col_b].nunique().max() == 1
    
    # Check if B->A mapping is one-to-one
    b_to_a_unique = df.groupby(col_b)[col_a].nunique().max() == 1
    
    # Both conditions must be true
    return a_to_b_unique and b_to_a_unique

# Example usage
result = is_bijective(df, 'o_uri', 'obj_id')
print(f"Columns are bijective: {result}")

result = is_bijective(df, 'o_aliases', 'possible_answers')
print(f"Columns are bijective: {result}")

Columns are bijective: True
Columns are bijective: False


In [24]:
# Filter out all obj_ids that have only 1 row
# Keep only obj_ids that appear more than once
# Count how many times each obj_id appears
obj_id_counts = df['obj_id'].value_counts()

# Get obj_ids that appear more than once
obj_ids_multiple = obj_id_counts[obj_id_counts > 1].index

# Filter the dataframe to keep only rows with obj_ids that appear multiple times
df_filtered = df[df['obj_id'].isin(obj_ids_multiple)]

print(f"Original dataframe shape: {df.shape}")
print(f"Filtered dataframe shape: {df_filtered.shape}")
print(f"Removed {df.shape[0] - df_filtered.shape[0]} rows with unique obj_ids")

# Show some statistics
print(f"\nOriginal obj_id counts:")
print(f"  Total unique obj_ids: {df['obj_id'].nunique()}")
print(f"  obj_ids with count = 1: {(obj_id_counts == 1).sum()}")
print(f"  obj_ids with count > 1: {(obj_id_counts > 1).sum()}")

print(f"\nFiltered obj_id counts:")
filtered_counts = df_filtered['obj_id'].value_counts()
print(f"  Total unique obj_ids: {df_filtered['obj_id'].nunique()}")
print(f"  Min count per obj_id: {filtered_counts.min()}")
print(f"  Max count per obj_id: {filtered_counts.max()}")

# Update the main dataframe
df = df_filtered

print(f"After removing: {len(df)}")


Original dataframe shape: (11981, 17)
Filtered dataframe shape: (7195, 17)
Removed 4786 rows with unique obj_ids

Original obj_id counts:
  Total unique obj_ids: 6399
  obj_ids with count = 1: 4786
  obj_ids with count > 1: 1613

Filtered obj_id counts:
  Total unique obj_ids: 1613
  Min count per obj_id: 2
  Max count per obj_id: 327
After removing: 7195


In [30]:
relation_counts = df['prop'].value_counts()
print("Current relation distribution:")
print(relation_counts)

# Sample equally from each relation (comment out Option A if using this)
max_per_relation = 500  # Adjust as needed
df_balanced = df.groupby('prop').apply(
    lambda x: x.sample(min(len(x), max_per_relation), random_state=42)
).reset_index(drop=True)

df = df_balanced
print(f"After balancing relations: {len(df)}")
print(df['prop'].value_counts())

Current relation distribution:
prop
genre             1325
director           879
screenwriter       811
country            773
producer           643
sport              526
author             519
occupation         487
composer           430
religion           276
place of birth     251
capital            128
father              86
color               32
mother              29
Name: count, dtype: int64
After balancing relations: 5219
prop
author            500
country           500
director          500
genre             500
producer          500
screenwriter      500
sport             500
occupation        487
composer          430
religion          276
place of birth    251
capital           128
father             86
color              32
mother             29
Name: count, dtype: int64


/tmp/ipykernel_398840/3717894448.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_balanced = df.groupby('prop').apply(


In [33]:
df['obj_length'] = df['obj'].str.len()
df['subj_length'] = df['subj'].str.len()

# Remove extremely short/long subjects and objects
df = df[
    (df['obj_length'] >= 2) & (df['obj_length'] <= 50) &
    (df['subj_length'] >= 2) & (df['subj_length'] <= 50)
]

print(f"After length filtering: {len(df)}")


After length filtering: 5193


In [37]:
# Remove subjects/objects that are just numbers or single characters
df =df[~df['obj'].str.match(r'^\d+$')]  # Not just numbers
df = df[~df['subj'].str.match(r'^\d+$')]  # Not just numbers
print(f"After removing numeric-only entries: {len(df)}")

After removing numeric-only entries: 5170


In [40]:
# If dataset is still too large, sample 
max_total_size = 5000 

if len(df) > max_total_size:
    # Sample proportionally from each relation
    df = df.groupby('prop').apply(
        lambda x: x.sample(min(len(x), max_total_size // df['prop'].nunique()), random_state=42)
    ).reset_index(drop=True)
print(f"After final sampling: {len(df)}")



After final sampling: 3797


In [41]:
df

,subj,prop,id,obj,subj_id,prop_id,obj_id,s_aliases,o_aliases,s_uri,o_uri,s_wiki_title,o_wiki_title,s_pop,o_pop,question,possible_answers,obj_length,subj_length
0,Sonnet 109,author,2649005,William Shakespeare,1141555,484,2310487,"[""Sonnets. 109""]","[""Shakespeare"",""The Bard"",""The Bard of Avon"",""...",http://www.wikidata.org/entity/Q3490513,http://www.wikidata.org/entity/Q692,Sonnet 109,William Shakespeare,355,439673,Who is the author of Sonnet 109?,"[""William Shakespeare"", ""Shakespeare"", ""The Ba...",19,10
1,Let's Not,author,1000986,Isaac Asimov,427743,484,1144266,[],"[""Isaak Osimov"",""Paul French"",""Asimov"",""Isaak ...",http://www.wikidata.org/entity/Q16386665,http://www.wikidata.org/entity/Q34981,Let's Not,Isaac Asimov,85,266562,Who is the author of Let's Not?,"[""Isaac Asimov"", ""Isaak Osimov"", ""Paul French""...",12,9
2,A Boy and His Dog,author,1161343,Harlan Ellison,514333,484,1026086,[],"[""Harlan Jay Ellison""]",http://www.wikidata.org/entity/Q17049896,http://www.wikidata.org/entity/Q311253,A Boy and His Dog,Harlan Ellison,6601,28055,Who is the author of A Boy and His Dog?,"[""Harlan Ellison"", ""Harlan Jay Ellison""]",14,17
3,Disclosure,author,155638,Michael Crichton,62758,484,534524,[],"[""John Michael Crichton"",""Michael Douglas"",""Je...",http://www.wikidata.org/entity/Q11026841,http://www.wikidata.org/entity/Q172140,Disclosure (novel),Michael Crichton,2873,77167,Who is the author of Disclosure?,"[""Michael Crichton"", ""John Michael Crichton"", ...",16,10
4,Black Orchids,author,682254,Rex Stout,288415,484,1107103,[],"[""Rex Todhunter Stout""]",http://www.wikidata.org/entity/Q15091297,http://www.wikidata.org/entity/Q337351,Black Orchids (novella),Rex Stout,147,4213,Who is the author of Black Orchids?,"[""Rex Stout"", ""Rex Todhunter Stout""]",9,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3792,Louis Perrée,sport,1993158,fencing,874741,560,124061,"[""Louis Perree""]",[],http://www.wikidata.org/entity/Q2599319,http://www.wikidata.org/entity/Q12100,Louis Perrée,Fencing,46,34201,What sport does Louis Perrée play?,"[""fencing""]",7,12
3793,Diego Polenta,sport,1926922,association football,846981,560,920845,"[""Diego Fabi\u00e1n Polenta Musetti"",""Diego Fa...","[""football"",""soccer""]",http://www.wikidata.org/entity/Q2499709,http://www.wikidata.org/entity/Q2736,Diego Polenta,Association football,411,197767,What sport does Diego Polenta play?,"[""association football"", ""football"", ""soccer""]",20,13
3794,Giorgio Cagnotto,sport,5416039,diving,2408155,560,2658403,"[""Franco Giorgio Cagnotto""]",[],http://www.wikidata.org/entity/Q716469,http://www.wikidata.org/entity/Q7735,Giorgio Cagnotto,Diving (sport),116,14668,What sport does Giorgio Cagnotto play?,"[""diving""]",6,16
3795,David Homoláč,sport,1611841,association football,718101,560,920845,"[""David Homolac""]","[""football"",""soccer""]",http://www.wikidata.org/entity/Q2044032,http://www.wikidata.org/entity/Q2736,David Homoláč,Association football,31,197767,What sport does David Homoláč play?,"[""association football"", ""football"", ""soccer""]",20,13


In [ ]:
df.to_csv("data/popqa_filtered.csv", index=False)

### Construct facts JSONL

In [43]:
condition_data = df.sort_values(by="s_pop", ascending=False).head(100).reset_index(drop=True)
condition_data

,subj,prop,id,obj,subj_id,prop_id,obj_id,s_aliases,o_aliases,s_uri,o_uri,s_wiki_title,o_wiki_title,s_pop,o_pop,question,possible_answers,obj_length,subj_length
0,United States of America,capital,2266274,"Washington, D.C.",988513,422,2024419,"[""the United States of America"",""America"",""U.S...","[""Washington"",""Washington DC"",""Washington, DC""...",http://www.wikidata.org/entity/Q30,http://www.wikidata.org/entity/Q61,United States,"Washington, D.C.",1629691,283465,What is the capital of United States of America?,"[""Washington, D.C."", ""Washington"", ""Washington...",16,24
1,India,capital,5015775,New Delhi,2215085,422,2938512,"[""Bharat"",""Hindustan"",""Bharatvarsh"",""in"",""IN"",...","[""New Delhi district"",""Nayi Dilli""]",http://www.wikidata.org/entity/Q668,http://www.wikidata.org/entity/Q987,India,New Delhi,1301086,128193,What is the capital of India?,"[""New Delhi"", ""New Delhi district"", ""Nayi Dilli""]",9,5
2,Elizabeth II,religion,6515620,Anglicanism,2925405,106,2145926,"[""Queen Elizabeth II"",""Elizabeth Alexandra Mar...","[""Anglicanism, Anglican Church""]",http://www.wikidata.org/entity/Q9682,http://www.wikidata.org/entity/Q6423963,Elizabeth II,Anglicanism,1291432,56744,What is the religion of Elizabeth II?,"[""Anglicanism"", ""Anglicanism, Anglican Church""]",11,12
3,United Kingdom,capital,593981,London,242916,422,2840765,"[""\ud83c\uddec\ud83c\udde7"",""UK"",""United Kingd...","[""London, UK"",""London, United Kingdom"",""London...",http://www.wikidata.org/entity/Q145,http://www.wikidata.org/entity/Q84,United Kingdom,London,1066107,523018,What is the capital of United Kingdom?,"[""London"", ""London, UK"", ""London, United Kingd...",6,14
4,Halloween,composer,1728085,John Carpenter,763729,639,2912281,[],"[""John Howard Carpenter""]",http://www.wikidata.org/entity/Q221103,http://www.wikidata.org/entity/Q95008,Halloween (1978 film),John Carpenter,933638,487945,Who was the composer of Halloween?,"[""John Carpenter"", ""John Howard Carpenter""]",14,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Yemen,capital,6224331,Sanaa,2808511,422,837811,"[""Republic of Yemen"",""ye"",""Rep. Yemen"",""\ud83c...","[""Sanaa"",""Sana"",""\u1e62an\u2018\u0101\u2019"",""...",http://www.wikidata.org/entity/Q805,http://www.wikidata.org/entity/Q2471,Yemen,Sanaa,148099,16308,What is the capital of Yemen?,"[""Sanaa"", ""Sanaa"", ""Sana"", ""Ṣan‘ā’"", ""Sana‘a"",...",5,5
96,Mark Twain,religion,5489994,atheism,2443351,106,2359563,"[""Samuel Langhorne Clemens"",""Samuel L. Clemens...","[""unbelief""]",http://www.wikidata.org/entity/Q7245,http://www.wikidata.org/entity/Q7066,Mark Twain,Atheism,146014,82132,What is the religion of Mark Twain?,"[""atheism"", ""unbelief""]",7,10
97,Atlético de Madrid,color,6340616,red,2858886,472,1036579,"[""Club Atl\u00e9tico de Madrid, SAD"",""Atletico...","[""red color""]",http://www.wikidata.org/entity/Q8701,http://www.wikidata.org/entity/Q3142,Atlético Madrid,Red,144999,51772,What color is Atlético de Madrid?,"[""red"", ""red color""]",3,18
98,Quebec,capital,1259497,Quebec City,561022,422,747990,"[""Qu\u00e9bec"",""QC"",""Province of Quebec"",""Queb...","[""Quebec"",""Ville de Qu\u00e9bec"",""Qu\u00e9bec""...",http://www.wikidata.org/entity/Q176,http://www.wikidata.org/entity/Q2145,Quebec,Quebec City,142909,48989,What is the capital of Quebec?,"[""Quebec City"", ""Quebec"", ""Ville de Québec"", ""...",11,6


In [45]:
df["o_aliases"] = df["o_aliases"].apply(literal_eval)
df["possible_answers"] = df["possible_answers"].apply(literal_eval)
df["s_aliases"] = df["s_aliases"].apply(literal_eval)

<unknown>:1: SyntaxWarning: invalid escape sequence '\/'
<unknown>:1: SyntaxWarning: invalid escape sequence '\/'
<unknown>:1: SyntaxWarning: invalid escape sequence '\/'
<unknown>:1: SyntaxWarning: invalid escape sequence '\/'
<unknown>:1: SyntaxWarning: invalid escape sequence '\/'
<unknown>:1: SyntaxWarning: invalid escape sequence '\/'
<unknown>:1: SyntaxWarning: invalid escape sequence '\/'
<unknown>:1: SyntaxWarning: invalid escape sequence '\/'
<unknown>:1: SyntaxWarning: invalid escape sequence '\/'
<unknown>:1: SyntaxWarning: invalid escape sequence '\/'
<unknown>:1: SyntaxWarning: invalid escape sequence '\/'
<unknown>:1: SyntaxWarning: invalid escape sequence '\/'
<unknown>:1: SyntaxWarning: invalid escape sequence '\/'
<unknown>:1: SyntaxWarning: invalid escape sequence '\/'
<unknown>:1: SyntaxWarning: invalid escape sequence '\/'
<unknown>:1: SyntaxWarning: invalid escape sequence '\/'
<unknown>:1: SyntaxWarning: invalid escape sequence '\/'
<unknown>:1: SyntaxWarning: inv

In [46]:
facts = []
failed_facts = []
for i, row in df.iterrows():
    relation = row["prop"]
    possible_obj_ctxs = df[df["prop"] == relation]["obj"].unique()
    obj_ctx = row["obj"]
    obj_prior_possibilities = [row["obj"]] + row["o_aliases"]

    available_alternatives = [obj for obj in possible_obj_ctxs if obj not in obj_prior_possibilities]
    if len(available_alternatives) == 0:
        print(f"Warning: No alternative objects found for {row['subj']} - {relation}")
        failed_facts.append({"row_index": i, "reason": "no_alternatives", "fact": row.to_dict()})
        continue
    
    while obj_ctx in obj_prior_possibilities:
        # obj_ctx = random.choice(possible_obj_ctxs)
        obj_ctx = random.choice(available_alternatives)
        obj_ctx_aliases = df[df["obj"] == obj_ctx]["o_aliases"].iloc[0]
    extra_info_obj_ctxs_data = df[df["obj"] == obj_ctx]
    extra_info_obj_ctx_data = extra_info_obj_ctxs_data.sample(n=1)
    extra_info_obj_ctx = f"is the {extra_info_obj_ctx_data['prop'].item()} of {extra_info_obj_ctx_data['subj'].item()}"
    
    condition_row = condition_data.sample(n=1)
    condition = f"{condition_row['obj'].item()} is the {condition_row['prop'].item()} of {condition_row['subj'].item()}"
    counterfactual_condition = f"{condition_row['obj'].item()} were the {condition_row['prop'].item()} of {condition_row['subj'].item()}"
    facts.append(
        {
            "subject": row["subj"],
            "relation": row["prop"],
            "object_ctx": obj_ctx,
            "object_pri": row["obj"],
            "object_ctx_aliases": obj_ctx_aliases,
            "object_prior_aliases": row["o_aliases"],
            "possible_prior_answers": row["possible_answers"],
            "obj_prior_id": row["obj_id"],
            "subj_pop": row["s_pop"],
            "obj_pop": row["o_pop"],
            "subject_relation": f"the {row['prop']} of {row['subj']}",
            "extra_info_obj_ctx": extra_info_obj_ctx,
            "condition": condition,
            "counterfactual_condition": counterfactual_condition,
            "condition_index": int(condition_row.index[0])
        }
    )

print(f"Successfully created: {len(facts)} facts")
print(f"Failed to create: {len(failed_facts)} facts")


Successfully created: 3797 facts
Failed to create: 0 facts


In [47]:
# ==== FINAL SUMMARY AND QUALITY CHECK ====
print("\n" + "="*50)
print("FINAL DATASET SUMMARY")
print("="*50)

facts_df = pd.DataFrame(facts)

print(f"Total facts generated: {len(facts_df)}")
print(f"Unique subjects: {facts_df['subject'].nunique()}")
print(f"Unique relations: {facts_df['relation'].nunique()}")
print(f"Unique correct objects: {facts_df['object_pri'].nunique()}")
print(f"Unique wrong objects: {facts_df['object_ctx'].nunique()}")

print(f"\nRelation distribution:")
relation_dist = facts_df['relation'].value_counts()
print(relation_dist)

print(f"\nSample facts for quality check:")
print("-" * 40)
for i, fact in enumerate(facts_df.sample(5, random_state=42).to_dict('records')):
    print(f"\nSample {i+1}:")
    print(f"  Question: What is {fact['subject_relation']}?")
    print(f"  Correct: {fact['object_pri']}")
    print(f"  Wrong: {fact['object_ctx']} (which {fact['extra_info_obj_ctx']})")
    print(f"  Condition: {fact['condition']}")

# Check for potential quality issues
print(f"\nQuality checks:")
print(f"  Facts where correct=wrong: {sum(facts_df['object_pri'] == facts_df['object_ctx'])}")
print(f"  Facts with empty wrong answer: {sum(facts_df['object_ctx'].isna() | (facts_df['object_ctx'] == ''))}")
print(f"  Average subject popularity: {facts_df['subj_pop'].mean():.1f}")
print(f"  Average object popularity: {facts_df['obj_pop'].mean():.1f}")


FINAL DATASET SUMMARY
Total facts generated: 3797
Unique subjects: 3553
Unique relations: 15
Unique correct objects: 1315
Unique wrong objects: 1123

Relation distribution:
relation
author            333
composer          333
country           333
director          333
genre             333
occupation        333
producer          333
screenwriter      333
sport             333
religion          275
place of birth    250
capital           128
father             86
color              32
mother             29
Name: count, dtype: int64

Sample facts for quality check:
----------------------------------------

Sample 1:
  Question: What is the author of Invasion?
  Correct: Dean Koontz
  Wrong: Stephen King (which is the author of Trucks)
  Condition: Santiago is the capital of Chile

Sample 2:
  Question: What is the religion of Ali Ibn Ibrahim Qomi?
  Correct: Shia Islam
  Wrong: atheism (which is the religion of Mark Lindley)
  Condition: Rome is the capital of Italy

Sample 3:
  Questi

In [48]:
facts[:10]

[{'subject': 'Sonnet 109',
  'relation': 'author',
  'object_ctx': 'Upton Sinclair',
  'object_pri': 'William Shakespeare',
  'object_ctx_aliases': ['Upton Beall Sinclair',
   'Clarke Fitch',
   'Frederick Garrison',
   'Arthur Stirling',
   'Upton Sinclair Jr.'],
  'object_prior_aliases': ['Shakespeare',
   'The Bard',
   'The Bard of Avon',
   'William Shakspere',
   'Swan of Avon',
   'Bard of Avon',
   'Shakespere',
   'Shakespear',
   'Shakspeare',
   'Shackspeare',
   'William Shake‐ſpeare'],
  'possible_prior_answers': ['William Shakespeare',
   'Shakespeare',
   'The Bard',
   'The Bard of Avon',
   'William Shakspere',
   'Swan of Avon',
   'Bard of Avon',
   'Shakespere',
   'Shakespear',
   'Shakspeare',
   'Shackspeare',
   'William Shake‐ſpeare'],
  'obj_prior_id': 2310487,
  'subj_pop': 355,
  'obj_pop': 439673,
  'subject_relation': 'the author of Sonnet 109',
  'extra_info_obj_ctx': 'is the author of The Jungle',
  'condition': 'Toronto is the capital of Ontario',
  'co

In [49]:
write_dicts_one_by_one(facts, "data/popqa_filtered_v2.jsonl")

Writing 3797 records to data/popqa_filtered_v2.jsonl (one by one)
Error writing item: 'utf-8' codec can't encode characters in position 141-144: surrogates not allowed
{'subject': 'Genoa', 'relation': 'country', 'object_ctx': 'Hungary', 'object_pri': 'United States of America', 'object_ctx_aliases': ['hu', '\ud83c\udded\ud83c\uddfa', 'HUN'], 'object_prior_aliases': ['the United States of America', 'America', 'U.S.A.', 'USA', 'U.S.', 'US', 'the US', 'the USA', 'US of A', 'the United States', 'U. S. A.', 'U. S.', 'the States', 'the U.S.', "'Merica", 'U.S', 'United States', "'Murica"], 'possible_prior_answers': ['United States of America', 'the United States of America', 'America', 'U.S.A.', 'USA', 'U.S.', 'US', 'the US', 'the USA', 'US of A', 'the United States', 'U. S. A.', 'U. S.', 'the States', 'the U.S.', "'Merica", 'U.S', 'United States', "'Murica"], 'obj_prior_id': 988513, 'subj_pop': 64, 'obj_pop': 1629691, 'subject_relation': 'the country of Genoa', 'extra_info_obj_ctx': 'is the 

### Scratch

In [13]:
# # Find rows where o_aliases and possible_answers are not bijective
# print("=== ANALYZING NON-BIJECTIVE RELATIONSHIPS ===\n")

# # Check if columns exist
# if 'o_aliases' not in df.columns:
#     print("Error: 'o_aliases' column not found in dataframe")
#     print("Available columns:", df.columns.tolist())
# elif 'possible_answers' not in df.columns:
#     print("Error: 'possible_answers' column not found in dataframe") 
#     print("Available columns:", df.columns.tolist())
# else:
#     # First, let's check overall bijection status
#     result = is_bijective(df, 'o_aliases', 'possible_answers')
#     print(f"Overall bijection status: {result}")
    
#     if not result:
#         print("\n=== FINDING BIJECTION VIOLATIONS ===\n")
        
#         # Convert to string representation for grouping (since these might be lists)
#         df_str = df.copy()
#         df_str['o_aliases_str'] = df_str['o_aliases'].astype(str)
#         df_str['possible_answers_str'] = df_str['possible_answers'].astype(str)
        
#         # Check o_aliases -> possible_answers violations (one o_aliases maps to multiple possible_answers)
#         alias_to_answers = df_str.groupby('o_aliases_str')['possible_answers_str'].nunique()
#         alias_violations = alias_to_answers[alias_to_answers > 1]
        
#         if len(alias_violations) > 0:
#             print(f"1. ALIASES -> ANSWERS VIOLATIONS: {len(alias_violations)} aliases map to multiple answer sets")
#             print("   (Same alias appears with different possible_answers)\n")
            
#             for alias_str in alias_violations.index[:5]:  # Show first 5
#                 matching_rows = df_str[df_str['o_aliases_str'] == alias_str]
#                 print(f"   Alias: {matching_rows.iloc[0]['o_aliases']}")
#                 print(f"   Maps to {len(matching_rows)} different answer sets:")
#                 for i, (_, row) in enumerate(matching_rows.iterrows()):
#                     print(f"     {i+1}. {row['possible_answers']} (subj: {row['subj']}, prop: {row['prop']})")
#                 print()
        
#         # Check possible_answers -> o_aliases violations (one possible_answers maps to multiple o_aliases)
#         answers_to_alias = df_str.groupby('possible_answers_str')['o_aliases_str'].nunique()
#         answer_violations = answers_to_alias[answers_to_alias > 1]
        
#         if len(answer_violations) > 0:
#             print(f"2. ANSWERS -> ALIASES VIOLATIONS: {len(answer_violations)} answer sets map to multiple aliases")
#             print("   (Same possible_answers appears with different o_aliases)\n")
            
#             for answers_str in answer_violations.index[:5]:  # Show first 5
#                 matching_rows = df_str[df_str['possible_answers_str'] == answers_str]
#                 print(f"   Possible answers: {matching_rows.iloc[0]['possible_answers']}")
#                 print(f"   Maps to {len(matching_rows)} different aliases:")
#                 for i, (_, row) in enumerate(matching_rows.iterrows()):
#                     print(f"     {i+1}. {row['o_aliases']} (subj: {row['subj']}, prop: {row['prop']})")
#                 print()
        
#         # Show some sample problematic rows
#         print("=== SAMPLE PROBLEMATIC ROWS ===\n")
        
#         if len(alias_violations) > 0:
#             sample_alias = alias_violations.index[0]
#             problem_rows = df_str[df_str['o_aliases_str'] == sample_alias]
#             print("Sample rows with same o_aliases but different possible_answers:")
#             print(problem_rows[['subj', 'obj', 'prop', 'o_aliases', 'possible_answers']].to_string())
#             print()
        
#         if len(answer_violations) > 0:
#             sample_answer = answer_violations.index[0]
#             problem_rows = df_str[df_str['possible_answers_str'] == sample_answer]
#             print("Sample rows with same possible_answers but different o_aliases:")
#             print(problem_rows[['subj', 'obj', 'prop', 'o_aliases', 'possible_answers']].to_string())
    
#     else:
#         print("The columns are bijective - no violations found!")


## Add authority belief

In [54]:
with open("data/popqa_filtered_v2.jsonl", 'r') as f:
    facts = [json.loads(line) for line in f]

# Possible sources
authority_sources = [
    "Wikipedia", 
    "Encyclopedia Britannica", 
    "the official database",
    "government records",
    "the academic encyclopedia",
    "historical documents",
    "official records",
    "the reference manual",
    "scholarly sources",
    "archival records"
]

belief_sources = [
    "my professor",
    "my colleague", 
    "my friend",
    "the expert",
    "my teacher",
    "the researcher",
    "my advisor",
    "the specialist",
    "John",
    "Sarah",
    "the historian",
    "my sister",
    "the curator",
    "the scholar"
]

# Add sources to each fact
enhanced_facts = []
for fact in facts:
    # Create enhanced fact with additional source fields
    enhanced_fact = fact.copy()
    
    # Add random authority and belief sources
    enhanced_fact["authority_source"] = random.choice(authority_sources)
    enhanced_fact["belief_source"] = random.choice(belief_sources)
    
    enhanced_facts.append(enhanced_fact)


# Save enhanced facts
with open("data/popqa_filtered_v2_enhanced.jsonl", 'w') as f:
    for fact in enhanced_facts:
        f.write(json.dumps(fact) + '\n')

In [55]:
print("\nSample enhanced fact:")
sample = enhanced_facts[0]
print(f"Subject: {sample['subject']}")
print(f"Relation: {sample['relation']}")
print(f"Object: {sample['object_pri']}")
print(f"Authority source: {sample['authority_source']}")
print(f"Belief source: {sample['belief_source']}")

print(f"\nNow you can generate templates like:")
print(f"- Authority: 'According to {sample['authority_source']}, the {sample['relation']} of {sample['subject']} is {sample['object_ctx']}.'")
print(f"- Belief: '{sample['belief_source']} believes that the {sample['relation']} of {sample['subject']} is {sample['object_ctx']}.'")


Sample enhanced fact:
Subject: Sonnet 109
Relation: author
Object: William Shakespeare
Authority source: Encyclopedia Britannica
Belief source: my sister

Now you can generate templates like:
- Authority: 'According to Encyclopedia Britannica, the author of Sonnet 109 is Upton Sinclair.'
- Belief: 'my sister believes that the author of Sonnet 109 is Upton Sinclair.'


In [ ]:


# Save enhanced facts
with open("/../data/popqa_filtered_v2_enhanced.jsonl", 'w') as f:
    for fact in enhanced_facts:
        f.write(json.dumps(fact) + '\n')

print(f"Enhanced {len(enhanced_facts)} facts with authority and belief sources")

# Show sample
print("\nSample enhanced fact:")
sample = enhanced_facts[0]
print(f"Subject: {sample['subject']}")
print(f"Relation: {sample['relation']}")
print(f"Object: {sample['object_pri']}")
print(f"Authority source: {sample['authority_source']}")
print(f"Belief source: {sample['belief_source']}")

print(f"\nNow you can generate templates like:")
print(f"- Authority: 'According to {sample['authority_source']}, the {sample['relation']} of {sample['subject']} is {sample['object_ctx']}.'")
print(f"- Belief: '{sample['belief_source']} believes that the {sample['relation']} of {sample['subject']} is {sample['object_ctx']}.'")

FileNotFoundError: [Errno 2] No such file or directory: '/../data/popqa_filtered_v2.jsonl'